In [374]:
from bokeh.plotting import figure, output_file, show, output_notebook,ColumnDataSource
from bokeh.models import HoverTool
from bokeh.colors import RGB
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter,)
from bokeh.transform import transform
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib. pyplot as plt
from scipy import optimize


In [394]:
Data1 = pd.read_csv ('Data1.csv')

In [395]:
def CreateScatterGraph(xdata,ydata,title,xaxis,yaxis):
    fig = px.scatter(x=xdata, y=ydata)
    fig.update_layout(
        title= title,
        xaxis_title=xaxis,
        yaxis_title=yaxis,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f")
        )
    fig.show()

In [396]:
def CreateScatterGraphWithFitPoly(xdata,ydata,title,xaxis,yaxis,polyorder):
    
    fig.add_trace(go.Scatter(
        x=xdata, y=ydata,
        name="Data Points",
        mode='markers',
        marker_color='rgba(152, 0, 0, .8)'
    ))
    #Creating the Fit
    p = np.poly1d(np.polyfit(xdata,ydata,polyorder))
    xp = np.linspace(1, 8, 500)

    fig.add_trace(go.Scatter(
        x=xp, y=p(xp),
        name="Fit",
        mode ='lines'
    ))
    
    
    fig.update_layout(
        title= title,
        xaxis_title=xaxis,
        yaxis_title=yaxis,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f")
        )
    fig.show()

In [397]:
CreateScatterGraph(Data1.Time,Data1.X,"X co-ord Vs Time","time|s","X|m")

In [398]:
CreateScatterGraph(Data1.Time,Data1.Y,"Y co-ord Vs Time","time|s","Y|m")

In [399]:
CreateScatterGraph(Data1.X,Data1.Y,"Y co-ord Vs X","X|m","Y|m")

In [400]:
def TwoScatterOverlay(xaxis,y1,y2,name1,name2,title):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=xaxis, y=y1,
        name=name1,
        mode='markers',
        marker_color='rgba(152, 0, 0, .8)'
    ))

    fig.add_trace(go.Scatter(
        x=xaxis, y=y2,
        name=name2,
        marker_color='rgba(255, 182, 193, .9)'
    ))
    fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
    fig.update_layout(title=title,
                  yaxis_zeroline=False, xaxis_zeroline=False)


    fig.show()

In [401]:
TwoScatterOverlay(Data1.Time,Data1.X,Data1.Y,"x co-ords","y co-ords","X and Y Co-ords Against Time")

In [402]:
TwoScatterOverlay(Data1.Time,Data1['X Velocity'],Data1['Y Velocity'],"X Velocity","Y Velocity","X and Y Velocity Against Time")

In [403]:
#Velocity Vector Over Time
v_vector = np.sqrt(np.square(Data1['X Velocity']) + np.square(Data1['Y Velocity']))

Data1.insert(5, "V_vector", v_vector, True) 

In [404]:
#KE over Time
KE = 1/2 * np.square(Data1['V_vector'])
Data1.insert(5, "KE", KE, True) 

In [405]:
CreateScatterGraph(Data1.Time,Data1.KE,"Total KE vs Time","time|s","KE|J")

In [406]:
#Not a good fit as polyfit doesent work well with sinisoidal waves
CreateScatterGraphWithFitPoly(Data1.Time,Data1.X,"X co-ord Vs Time","time|s","X|m",2)


In [407]:
#Using Scipy Curve Fit opto for bette sin wave fitting 

In [408]:
def test_func(x, a, b,c,d):
    return a * np.sin(b*2*np.pi*x+c) + d

params, params_covariance = optimize.curve_fit(test_func, Data1.Time, Data1.X, p0=[0.25,.35,0,0],
                                               bounds=[[0,0,-np.inf,-0.5],[0.5,.8,np.inf,0.5]])


In [409]:
def TwoScatterOverlayLine(x1,y1,x2,y2,name1,name2,title):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x1, y=y1,
        name=name1,
        mode='markers',
        marker_color='rgba(152, 0, 0, .8)'
    ))

    fig.add_trace(go.Scatter(
        x=x2, y=y2,
        name=name2,
        mode='lines'
    ))
    #fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
    fig.update_layout(title=title,
                  yaxis_zeroline=False, xaxis_zeroline=False)


    fig.show()

In [410]:
TwoScatterOverlayLine(Data1.Time,Data1.X,Data1.Time,
                      test_func(Data1.Time, params[0], params[1],params[2],params[3]),"X Co-ords",'Line of Best Fit','Curve Fit for the Curve')

In [411]:
#From this I was alble to determine the Frequency and the Amplitude and Phase change by looking at the Params
params

array([ 2.44097015e-01,  5.20749968e-01, -4.82425127e+00, -4.68275626e-03])

In [412]:
print("Amplitude:{} \nFrequency:{} \nPhase:{}\nY Offset:{}".format(params[0],params[1],params[2],params[3]))

Amplitude:0.24409701486875904 
Frequency:0.5207499680656724 
Phase:-4.824251271547738
Y Offset:-0.004682756262665598
